In [1]:
# IMPORTANT: Patch OpenAI for tracing BEFORE any deepeval imports
import openai
from deepeval.openai.patch import patch_openai
patch_openai(openai)

obsolete approach, resultet in one giant cluster as documents are all quite similar

In [ ]:
# from langchain_ollama import OllamaEmbeddings
# from sklearn.cluster import AgglomerativeClustering
# import numpy as np
# from typing import List, Dict
# import json
# from dotenv import load_dotenv
# load_dotenv()

# # Load filtered_document_paths from JSON file
# with open("filtered_document_paths.json", "r", encoding="utf-8") as f:
#     filtered_document_paths = json.load(f)["filtered_document_paths"]

# def create_rag_simulated_contexts(
#     document_paths: List[str],
#     context_size: int = 3,  # Match your RAG top-k
#     similarity_threshold: float = 0.3,
#     max_context_length: int = 6000  # Prevent overly large contexts
# ) -> List[Dict]:
#     """
#     Create contexts that simulate your RAG system's top-k retrieval
#     while ensuring semantic coherence through clustering
#     """
    
#     # Load documents
#     docs = []
#     doc_metadata = []
    
#     print(f"Loading and filtering {len(document_paths)} documents...")
#     for idx, path in enumerate(document_paths):
#         with open(path, 'r', encoding='utf-8') as f:
#             content = f.read().strip()
#             if len(content) > 100:  # Filter very short docs
#                 docs.append(content)
#                 doc_metadata.append({
#                     'path': path,
#                     'content': content,
#                     'length': len(content)
#                 })
#         # Print progress every 10 files or at the end
#         if (idx + 1) % 10 == 0 or (idx + 1) == len(document_paths):
#             print(f"  Processed {idx + 1}/{len(document_paths)} files", end="\r")
#     print(f"\nLoaded {len(docs)} documents after filtering.")

#     if len(docs) < context_size:
#         print(f"Warning: Only {len(docs)} documents available, less than context_size {context_size}")
#         return []
    
#     # Embed documents using OllamaEmbeddings with xbai-embed-large:335m
#     print("Embedding documents...")
#     ollama_model = OllamaEmbeddings(model="mxbai-embed-large:335m")
#     embeddings = []
#     for i, doc in enumerate(docs):
#         emb = ollama_model.embed_query(doc)
#         embeddings.append(emb)
#         # Print progress every 10 docs or at the end
#         if (i + 1) % 10 == 0 or (i + 1) == len(docs):
#             print(f"  Embedded {i + 1}/{len(docs)} documents", end="\r")
#     embeddings = np.array(embeddings)
#     print(f"\nEmbedding complete. Shape: {embeddings.shape}")

#     # Use AgglomerativeClustering for coherent groups
#     print("Clustering documents...")
#     clustering = AgglomerativeClustering(
#         n_clusters=None,
#         distance_threshold=1-similarity_threshold,  # Convert to distance
#         metric="cosine",
#         linkage="average"
#     )
#     labels = clustering.fit_predict(embeddings)
#     print(f"Clustering complete. Found {len(set(labels))} clusters.")

#     # Group documents by cluster
#     from collections import defaultdict
#     clusters = defaultdict(list)
#     for label, meta in zip(labels, doc_metadata):
#         clusters[label].append(meta)
    
#     # Create contexts from clusters (simulating RAG top-k behavior)
#     contexts = []
#     print("Constructing contexts from clusters...")
#     for label_idx, (label, cluster_docs) in enumerate(clusters.items()):
#         if len(cluster_docs) >= context_size:
#             # Sort by length (shorter first) for better context construction
#             cluster_docs.sort(key=lambda x: x['length'])
            
#             # Create multiple contexts from this cluster
#             for i in range(0, len(cluster_docs), context_size):
#                 context_docs = cluster_docs[i:i+context_size]
                
#                 # Check total length constraint
#                 total_length = sum(doc['length'] for doc in context_docs)
#                 if total_length > max_context_length:
#                     # If too long, use only the first few docs
#                     context_docs = context_docs[:context_size]
#                     total_length = sum(doc['length'] for doc in context_docs[:context_size])
                
#                 if len(context_docs) >= 2:  # Require at least 2 docs for multi-context
#                     context_content = []
#                     source_paths = []
                    
#                     for doc in context_docs:
#                         context_content.append(doc['content'])
#                         source_paths.append(doc['path'])
                    
#                     contexts.append({
#                         'content': context_content,
#                         'source_paths': source_paths,
#                         'cluster_id': label,
#                         'context_size': len(context_content),
#                         'total_length': total_length
#                     })
#         # Print progress every 5 clusters or at the end
#         if (label_idx + 1) % 5 == 0 or (label_idx + 1) == len(clusters):
#             print(f"  Processed {label_idx + 1}/{len(clusters)} clusters", end="\r")
#     print(f"\nContext construction complete. Generated {len(contexts)} contexts.")

#     return contexts

# # Usage example
# contexts = create_rag_simulated_contexts(
#     document_paths=filtered_document_paths,
#     context_size=3,  # Match your RAG top-k
#     similarity_threshold=0.3 # Note this is 1 - cosine similarity
# )

# # Convert to format expected by DeepEval
# deepeval_contexts = [ctx['content'] for ctx in contexts]

Loading and filtering 545 documents...
  Processed 545/545 files
Loaded 545 documents after filtering.
Embedding documents...
  Embedded 545/545 documents
Embedding complete. Shape: (545, 1024)
Clustering documents...
Clustering complete. Found 1 clusters.
Constructing contexts from clusters...
  Processed 1/1 clusters
Context construction complete. Generated 182 contexts.


alternative to above, because we only got 1 giant cluster and then arbitrary contexts...

```
This approach uses AgglomerativeClustering (keeping your existing code structure) but iterates over possible n_clusters values, picks the one with the highest silhouette score, and proceeds. It adds a bit of computation (clustering a few times), but for 545 docs, it's fast (~seconds on a typical machine). If the optimal ends up with some clusters >3, we can still slice them into multiple 3-doc contexts as before. We'll also allow 1- or 2-doc contexts to maximize output.
```


In [2]:
from langchain_ollama import OllamaEmbeddings
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score  # New import for quality metric
import numpy as np
from typing import List, Dict
import json
from dotenv import load_dotenv
load_dotenv()

# Load filtered_document_paths from JSON file
with open("filtered_document_paths.json", "r", encoding="utf-8") as f:
    filtered_document_paths = json.load(f)["filtered_document_paths"]

def create_rag_simulated_contexts(
    document_paths: List[str],
    context_size: int = 3,  # Match your RAG top-k
    similarity_threshold: float = 0.3,
    max_context_length: int = 6000  # Prevent overly large contexts
) -> List[Dict]:
    """
    Create contexts that simulate your RAG system's top-k retrieval
    while ensuring semantic coherence through clustering
    """
    
    # Load documents
    docs = []
    doc_metadata = []
    
    print(f"Loading and filtering {len(document_paths)} documents...")
    for idx, path in enumerate(document_paths):
        with open(path, 'r', encoding='utf-8') as f:
            content = f.read().strip()
            if len(content) > 100:  # Filter very short docs
                docs.append(content)
                doc_metadata.append({
                    'path': path,
                    'content': content,
                    'length': len(content)
                })
        # Print progress every 10 files or at the end
        if (idx + 1) % 10 == 0 or (idx + 1) == len(document_paths):
            print(f"  Processed {idx + 1}/{len(document_paths)} files", end="\r")
    print(f"\nLoaded {len(docs)} documents after filtering.")

    if len(docs) < context_size:
        print(f"Warning: Only {len(docs)} documents available, less than context_size {context_size}")
        return []
    
    # Embed documents using OllamaEmbeddings with mxbai-embed-large:335m
    print("Embedding documents...")
    ollama_model = OllamaEmbeddings(model="mxbai-embed-large:335m")
    embeddings = []
    for i, doc in enumerate(docs):
        emb = ollama_model.embed_query(doc)
        embeddings.append(emb)
        # Print progress every 10 docs or at the end
        if (i + 1) % 10 == 0 or (i + 1) == len(docs):
            print(f"  Embedded {i + 1}/{len(docs)} documents", end="\r")
    embeddings = np.array(embeddings)
    print(f"\nEmbedding complete. Shape: {embeddings.shape}")

    # Find optimal number of clusters using silhouette score
    print("Finding optimal number of clusters...")
    min_clusters = max(1, len(embeddings) // 5)  # Bias toward smaller clusters (~3-5 docs)
    max_clusters = len(embeddings) // 2
    best_score = -1
    best_n_clusters = min_clusters
    best_labels = None
    
    for n in range(min_clusters, max_clusters + 1, 5):  # Step by 5 for speed
        clustering = AgglomerativeClustering(
            n_clusters=n,
            metric="cosine",
            linkage="average"
        )
        labels = clustering.fit_predict(embeddings)
        if len(set(labels)) > 1:  # Silhouette requires at least 2 clusters
            score = silhouette_score(embeddings, labels, metric="cosine")
            if score > best_score:
                best_score = score
                best_n_clusters = n
                best_labels = labels
        print(f"  Tried n_clusters={n}, score={score if 'score' in locals() else 'N/A'}", end="\r")
    
    print(f"\nOptimal n_clusters: {best_n_clusters} (silhouette score: {best_score})")
    labels = best_labels  # Use the best one

    # Group documents by cluster
    from collections import defaultdict
    clusters = defaultdict(list)
    for label, meta in zip(labels, doc_metadata):
        clusters[label].append(meta)
    
    # Create contexts from clusters (simulating RAG top-k behavior)
    contexts = []
    print("Constructing contexts from clusters...")
    for label_idx, (label, cluster_docs) in enumerate(clusters.items()):
        if len(cluster_docs) >= 1:  # Allow 1+ docs (changed from >= context_size)
            # Sort by length (shorter first) for better context construction
            cluster_docs.sort(key=lambda x: x['length'])
            
            # Create multiple contexts from this cluster
            for i in range(0, len(cluster_docs), context_size):
                context_docs = cluster_docs[i:i+context_size]
                
                # Check total length constraint
                total_length = sum(doc['length'] for doc in context_docs)
                if total_length > max_context_length:
                    # If too long, use only the first few docs
                    context_docs = context_docs[:context_size]
                    total_length = sum(doc['length'] for doc in context_docs[:context_size])
                
                if len(context_docs) >= 1:  # Allow 1+ (changed from >=2)
                    context_content = []
                    source_paths = []
                    
                    for doc in context_docs:
                        context_content.append(doc['content'])
                        source_paths.append(doc['path'])
                    
                    contexts.append({
                        'content': context_content,
                        'source_paths': source_paths,
                        'cluster_id': label,
                        'context_size': len(context_content),
                        'total_length': total_length
                    })
        # Print progress every 5 clusters or at the end
        if (label_idx + 1) % 5 == 0 or (label_idx + 1) == len(clusters):
            print(f"  Processed {label_idx + 1}/{len(clusters)} clusters", end="\r")
    print(f"\nContext construction complete. Generated {len(contexts)} contexts.")

    return contexts

# Usage example
contexts = create_rag_simulated_contexts(
    document_paths=filtered_document_paths,
    context_size=3,  # Match your RAG top-k
    similarity_threshold=0.3 # Note this is 1 - cosine similarity
)

# Convert to format expected by DeepEval
deepeval_contexts = [ctx['content'] for ctx in contexts]

Loading and filtering 545 documents...
  Processed 545/545 files
Loaded 545 documents after filtering.
Embedding documents...
  Embedded 545/545 documents
Embedding complete. Shape: (545, 1024)
Finding optimal number of clusters...
  Tried n_clusters=269, score=0.15056691705621658
Optimal n_clusters: 199 (silhouette score: 0.1658298986627275)
Constructing contexts from clusters...
  Processed 199/199 clusters
Context construction complete. Generated 271 contexts.


In [3]:
contexts

[{'content': ["Source: https://satoshi.nakamotoinstitute.org/posts/bitcointalk/136/\nIt's too late now for feature changes to 0.3, but I'll add that to the post-0.3 to do list.Â\xa0 I never would have noticed that if you hadn't pointed it out.\nRe: Bitcoin in Ubuntu 10.04\nJune 22, 2010 at 16:39:43 UTC\nBitcoinTalk",
   'Source: https://satoshi.nakamotoinstitute.org/emails/bitcoin-list/31/\nFrom: | Satoshi Nakamoto | Subject: | [bitcoin-list] Alert: upgrade to bitcoin 0.3.6 | Date: | July 30, 2010 at 06:02:38 UTC\nPlease upgrade to 0.3.6 ASAP to get an important bugfix.\nSee the bitcoin.org homepage for download links.\n[bitcoin-list] Alert: upgrade to bitcoin 0.3.6\nbitcoin-list',
   'Source: https://satoshi.nakamotoinstitute.org/emails/bitcoin-list/34/\nFrom: | Satoshi Nakamoto | Subject: | [bitcoin-list] Bitcoin 0.3.19 is released | Date: | December 13, 2010 at 16:12:09 UTC\nThis is a minor release to add some DoS protection.\nChanges:\n- Added some DoS limits, though it\'s still fa

In [4]:
from deepeval.synthesizer import Synthesizer
from deepeval.synthesizer.config import EvolutionConfig, FiltrationConfig
from deepeval.models import GPTModel

LLM_MODEL = "gpt-4o-mini"
EMBEDDING_MODEL = "text-embedding-3-small"

llm = GPTModel(
    model=LLM_MODEL
)

# Configure synthesizer to avoid API timeouts
evolution_config = EvolutionConfig(num_evolutions=0)

filtration_cfg = FiltrationConfig(
    critic_model=llm, 
    synthetic_input_quality_threshold=0.5,
    max_quality_retries=1,  # Reduced retries to avoid timeout loops
)

synthesizer = Synthesizer(
    model=llm, 
    async_mode=True,  # Use sync mode to avoid hanging issues
    evolution_config=evolution_config,
    filtration_config=filtration_cfg,
    max_concurrent=16  # Reduced to avoid overwhelming the API
)

In [5]:
# Generate goldens (process in smaller batches to avoid overwhelming)
print(f"Generating goldens from {len(deepeval_contexts)} contexts...")

# Process in smaller batches to avoid memory/API issues  (It will otherwise get stuck at 0% with no error, or sometimes timeout error...)
batch_size = 10
all_goldens = []

for i in range(0, len(deepeval_contexts), batch_size):
    batch_contexts = deepeval_contexts[i:i+batch_size]
    print(f"Processing batch {i//batch_size + 1}/{(len(deepeval_contexts) + batch_size - 1)//batch_size} ({len(batch_contexts)} contexts)")

    try:
        batch_goldens = synthesizer.generate_goldens_from_contexts(
            contexts=batch_contexts,
            max_goldens_per_context=2,
            include_expected_output=True
        )
        all_goldens.extend(batch_goldens)
        print(f"  Generated {len(batch_goldens)} goldens from this batch")
    except Exception as e:
        print(f"  Error in batch {i//batch_size + 1}: {e}")
        continue

goldens = all_goldens
print(f"Total goldens generated: {len(goldens)}")


Output()

Generating goldens from 271 contexts...
Processing batch 1/28 (10 contexts)


Output()

  Generated 20 goldens from this batch
Processing batch 2/28 (10 contexts)


Output()

...

  Generated 20 goldens from this batch
Processing batch 26/28 (10 contexts)


Output()

Output()

  Generated 20 goldens from this batch
Processing batch 28/28 (1 contexts)


  Generated 2 goldens from this batch
Total goldens generated: 542


In [6]:
# Add metadata back to goldens
for i, golden in enumerate(goldens):
    if i < len(contexts):
        ctx_info = contexts[i]
        golden.additional_metadata = golden.additional_metadata or {}
        golden.additional_metadata.update({
            'source_documents': ctx_info['source_paths'],
            'cluster_id': ctx_info['cluster_id'],
            'context_size': ctx_info['context_size']
        })

In [7]:
goldens[0]

Golden(input='What is cryptocurrency?', actual_output=None, expected_output='Cryptocurrency is a form of electronic cash that enables direct online payments between parties without the need for a financial institution. It operates on a peer-to-peer network, utilizing technology like digital signatures and proof-of-work to ensure secure transactions and prevent issues like double-spending.', context=['Source: https://satoshi.nakamotoinstitute.org/quotes/cryptocurrency/\nCryptocurrency\nA purely peer-to-peer version of electronic cash would allow online payments to be sent directly from one party to another without going through a financial institution.\nOctober 31, 2008', 'Source: https://satoshi.nakamotoinstitute.org/quotes/bitcoin-design/\nBitcoin Design\nWe have proposed a system for electronic transactions without relying on trust. We started with the usual framework of coins made from digital signatures, which provides strong control of ownership, but is incomplete without a way to

In [8]:
def convert_numpy_types(obj):
    """Recursively convert numpy types to Python types for JSON serialization"""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convert_numpy_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    else:
        return obj

# Convert goldens to dictionaries for JSON serialization
serializable_goldens = []
for golden in goldens:
    golden_dict = {
        'input': golden.input,
        'expected_output': golden.expected_output,
        'context': golden.context,
        'source_file': getattr(golden, 'source_file', None),
        'additional_metadata': convert_numpy_types(getattr(golden, 'additional_metadata', {}))
    }
    serializable_goldens.append(golden_dict)

# Save to JSON
with open("multicontext_goldens.json", 'w', encoding='utf-8') as f:
    json.dump(convert_numpy_types(serializable_goldens), f, indent=2, ensure_ascii=False)

print(f"Saved {len(serializable_goldens)} goldens to multicontext_goldens.json")

Saved 542 goldens to multicontext_goldens.json
